# Proyecto Parcial — MLY0100
**Tema:** Predicción y Evaluación de Riesgo de Diabetes  
**Autor:** Antonio Sepúlveda  
**Fecha:** 2025


# 1. Entendimiento del Negocio

El objetivo de este análisis es explorar el **riesgo de diabetes** en pacientes utilizando el dataset PIMA Diabetes.
Esto permitirá entender mejor qué variables clínicas están más asociadas con el diagnóstico de diabetes
y preparar los datos para el pipeline de Machine Learning implementado con **Kedro**.

**Caso real:**  
En un contexto clínico, este tipo de análisis puede apoyar el **diagnóstico temprano**, la priorización de pacientes
y el diseño de estrategias preventivas para personas con alto riesgo de desarrollar diabetes.


En este notebook realizaremos:

- Carga del dataset original de diabetes (`data/01_raw/diabetes.csv`).  
- Revisión de estructura: número de filas y columnas.  
- Estadísticos descriptivos de variables numéricas.  
- Visualización de distribuciones.  
- Análisis de correlación entre variables.  
- Revisión de valores faltantes / valores 0 tratados como *missing*.  
- Ejemplo sencillo de tratamiento de valores faltantes y outliers.  
- Generación opcional de un dataset limpio para continuar el flujo de trabajo.


In [ ]:
# Código 1: Carga inicial de datos y vista general

import pandas as pd

# Ruta del dataset crudo (según estructura Kedro)
df = pd.read_csv('../data/01_raw/diabetes.csv')

# Mostrar las primeras filas
df.head()


In [ ]:
# Código 2: Número de filas y columnas + info básica del dataset

print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
df.info()


## Estadísticos descriptivos de variables numéricas

Aquí observamos la media, mediana, desviación estándar, valores mínimos y máximos de cada variable numérica.
Esto nos ayuda a entender la escala de cada variable y detectar posibles valores extremos.


In [ ]:
# Código 3: Estadísticas descriptivas

df.describe()


## Visualización de distribuciones

Los histogramas muestran la frecuencia de los valores en cada variable numérica.
Así podemos identificar si las distribuciones están sesgadas, si existen valores extremos (outliers)
o si las variables son aproximadamente normales.


In [ ]:
# Código 4: Histogramas de variables numéricas

import matplotlib.pyplot as plt
import seaborn as sns

df.hist(figsize=(12, 8), bins=20)
plt.tight_layout()
plt.show()


## Correlación entre variables

La matriz de correlación permite ver qué variables numéricas están más relacionadas entre sí
y con la variable objetivo `Outcome`.  
Valores cercanos a **1** o **-1** indican una relación lineal fuerte.


In [ ]:
# Código 5: Mapa de calor de correlación

plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de correlación - Dataset Diabetes')
plt.show()


## Balance de la variable objetivo `Outcome`

Revisamos cuántos pacientes tienen diagnóstico de diabetes (`1`) y cuántos no (`0`).
Esto es importante para detectar **desbalanceo de clases**, que puede afectar el rendimiento del modelo.


In [ ]:
# Código 6: Distribución de la variable objetivo

print(df['Outcome'].value_counts())

plt.figure(figsize=(4, 4))
sns.countplot(x='Outcome', data=df)
plt.title('Distribución de Outcome (0 = No Diabetes, 1 = Diabetes)')
plt.show()


## Revisión de valores faltantes y ceros sospechosos

En este dataset, algunos valores `0` en variables como **Glucose**, **BloodPressure**, **SkinThickness**,
**Insulin** y **BMI** pueden interpretarse como datos faltantes (clínicamente no tiene sentido tener
presión sanguínea o glucosa exactamente 0).

Primero revisamos valores faltantes explícitos (`NaN`) y luego contamos cuántos ceros hay en estas columnas.


In [ ]:
# Código 7: Valores faltantes y ceros sospechosos

print("Valores faltantes por columna:")
print(df.isnull().sum())

cols_zero_as_missing = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

print("\nConteo de ceros en columnas clínicas:")
for col in cols_zero_as_missing:
    print(f"{col}: {(df[col] == 0).sum()} ceros")


### Tratamiento de valores faltantes (ejemplo)

Como estrategia simple de preprocesamiento:

1. Reemplazamos los **ceros** en columnas clínicas por `NaN` para marcarlos como faltantes.  
2. Imputamos estos valores usando la **mediana**, que es robusta frente a outliers.

> Nota: El pipeline oficial de Kedro también realiza limpieza propia.  
> Este notebook muestra una versión *didáctica* del preprocesamiento.


In [ ]:
# Código 8: Reemplazo de ceros por NaN e imputación por mediana

import numpy as np
from sklearn.impute import SimpleImputer

# Copia del dataframe original para no perder la referencia
df_clean = df.copy()

# Paso 1: ceros -> NaN en columnas clínicas
for col in cols_zero_as_missing:
    df_clean[col] = df_clean[col].replace(0, np.nan)

print("Valores faltantes después de reemplazar ceros por NaN:")
print(df_clean.isnull().sum())

# Paso 2: imputación por mediana
imputer = SimpleImputer(strategy='median')
df_clean[cols_zero_as_missing] = imputer.fit_transform(df_clean[cols_zero_as_missing])

print("\nValores faltantes después de la imputación:")
print(df_clean.isnull().sum())


## Detección de outliers (ejemplo con `BMI`)

Los outliers (valores extremos) pueden afectar las métricas y el entrenamiento de los modelos.
Usamos el método del **rango intercuartílico (IQR)** para identificar valores extremos en la variable `BMI`.


In [ ]:
# Código 9: Detección de outliers en BMI usando IQR

Q1 = df_clean['BMI'].quantile(0.25)
Q3 = df_clean['BMI'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_bmi = df_clean[(df_clean['BMI'] < lower_bound) | (df_clean['BMI'] > upper_bound)]
print(f"Número de outliers en BMI: {outliers_bmi.shape[0]}")

# Boxplot para visualizar
plt.figure(figsize=(6, 4))
sns.boxplot(x=df_clean['BMI'])
plt.title('Boxplot de BMI (después de imputación)')
plt.show()


## Escalamiento de variables numéricas

Muchos modelos se benefician de que las variables numéricas estén en una escala similar.  
Aquí aplicamos **StandardScaler** (media 0, desviación estándar 1) a las variables de entrada
(sin incluir la variable objetivo `Outcome`).


In [ ]:
# Código 10: Escalamiento de variables numéricas

from sklearn.preprocessing import StandardScaler

feature_cols = [col for col in df_clean.columns if col != 'Outcome']

scaler = StandardScaler()
df_scaled = df_clean.copy()
df_scaled[feature_cols] = scaler.fit_transform(df_scaled[feature_cols])

df_scaled[feature_cols].describe().T


## Guardado opcional del dataset preprocesado

Podemos guardar una versión preprocesada del dataset para usarla en otros experimentos.

⚠️ **Advertencia:**  
El pipeline oficial de Kedro ya guarda sus propios archivos en `data/02_intermediate/`.  
Si no quieres sobreescribir nada, **no ejecutes** la siguiente celda o cambia el nombre del archivo.


In [ ]:
# Código 11 (opcional): Guardar dataset escalado para exploración

output_path = '../data/02_intermediate/diabetes_cleaned_eda.csv'
df_scaled.to_csv(output_path, index=False)
print(f"Archivo guardado en: {output_path}")
